- Make learner work generically

- Make it work with add_col=True
- Make it work with n_jobs > 1

In [1]:
#default_exp tabular_api

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
#export
from hypster.oo_hp import *
from hypster.hypster_prepare import *

import fastai2
from fastai2.tabular.all import *
from fastai2.metrics import *

from sklearn.model_selection import train_test_split

from copy import deepcopy

import optuna

In [4]:
#export
SEED = 42

# Read Data

In [5]:
#export
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

(#3) [Path('C:/Users/user/.fastai/data/adult_sample/adult.csv'),Path('C:/Users/user/.fastai/data/adult_sample/export.pkl'),Path('C:/Users/user/.fastai/data/adult_sample/models')]

In [6]:
#export
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [7]:
#export
df = df.sample(frac=0.1)

In [8]:
#export
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
dep_var = "salary"

In [9]:
#export
train_df, test_df = train_test_split(df, test_size=0.6, 
                                     random_state=SEED, 
                                     stratify=df[dep_var])

# Preprocessing

In [10]:
FillMissing = prepare(FillMissing)

In [11]:
fill_strategy = HpCategorical([FillStrategy.mode, 
                               FillStrategy.median, 
                               #FillStrategy.constant(5, fill)
                              ])

In [12]:
#imp = FillMissing(fill_strategy=fill_strategy, add_col=HpBool("add_col"))

In [13]:
imp = FillMissing(fill_strategy=fill_strategy, add_col=False)

In [14]:
#export
Normalize = prepare(Normalize)
norm = Normalize(mean=HpFloat(0.001, 10.4))

In [15]:
#norm = Normalize(mean=0.4)

In [16]:
#export
procs = [Categorify, imp, HpToggle(norm)]

<class 'hypster.hypster_prepare.HypsterPrepare'>
<class 'fastai2.data.transforms.Normalize'>
<class 'fastcore.transform._TfmMeta'>


In [ ]:
#if int, float, ..... Hp
#map
tabular_leaner(lr=1.5)
Float()


# DataBunch

In [17]:
import fastai2

In [18]:
TabularPandas = prepare(fastai2.tabular.data.TabularPandas)

In [19]:
#export
to = TabularPandas(train_df,
                   y_block = CategoryBlock(), 
                   y_names = dep_var,
                   splits = RandomSplitter()(range_of(train_df)),
                   cat_names = cat_names,
                   cont_names = cont_names,
                   procs = procs,
                   reduce_memory=False,
                   inplace=False
                  )

<class 'fastai2.tabular.core.FillMissing'>
<class 'fastai2.data.transforms.Normalize'>
<class 'hypster.hypster_prepare.HypsterPrepare'>
<class 'fastai2.data.transforms.Normalize'>
<class 'fastcore.transform._TfmMeta'>


In [20]:
#export
#dls = to.dataloaders(batch_size=2 ** HpInt("batch_size", 5, 9))
#dls = to.dataloaders(batch_size=HpInt("batch_size", 16, 128, 16))

In [21]:
#export
dls = to.dataloaders(batch_size=32)

# Learner

In [22]:
#export
cbs = [#TrackerCallback(monitor="roc_auc_score"), 
       TrackerCallback(monitor="accuracy"), 
       #ReduceLROnPlateau("roc_auc_score", patience=3)
      ]

In [23]:
#export
start_mom = HpFloat(0.85, 0.99)

In [24]:
#export
tabular_learner = prepare(tabular_learner)

In [25]:
#export
learner = tabular_learner(dls,
                          #metrics=RocAuc(),
                          metrics=accuracy,
                          opt_func=HpCategorical([Adam, SGD, QHAdam]),
                          layers=HpVarLenList(1, 4, HpInt(50, 300, 50, name="layer_size"), same_value=False),
                          cbs=cbs,
                          moms=(start_mom, start_mom-0.1, start_mom), 
                          wd_bn_bias=HpBool(),
                          )

dataloaders


# Optuna

In [26]:
lr = HpFloat(1e-5, 1e-1, log=True)

In [27]:
def lr_finder(trial, learner, lr_type="steep", **kwargs):
    #TODO: remove trial?
    learner = learner.sample(trial)
    lr_min, lr_steep = learner.lr_find(**kwargs)
    if lr_type == "steep":
        return lr_steep
    else:
        return lr_min * 0.1

In [28]:
lr = HpFunc(lr_finder, learner=learner, lr_type=HpCategorical(["min", "steep"]))

In [29]:
import datetime

In [30]:
#export
def run_learner(learner, fit_method, get_metric, n_trials=5): #learner
    class Objective():
        def __init__(self, fit_method, get_metric): #learner
            #self.learner   = learner
            self.fit_method = fit_method
            self.get_metric = get_metric
            
        def __call__(self, trial): 
            #learner = self.learner.sample(trial)
            self.fit_method.sample(trial)
            res = self.get_metric.sample(trial)
            #print(self.fit_method.base_call)
            #print(self.get_metric.base_call.base_call)
            print(res)
            return res
       
    objective = Objective(fit_method, get_metric) #learner
    optuna.logging.set_verbosity(0)
    pruner = optuna.pruners.NopPruner()
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    study = optuna.create_study(direction="maximize", study_name = now, pruner=pruner)
    
    #study.set_user_attr("names_db", {}) #TODO: Check what happens if n_jobs=-1
    #set_names(learner, fit_method, get_metric, study)
    
    study.optimize(objective, n_trials=n_trials, n_jobs=1, timeout=600)
    return study

In [31]:
#export
study = run_learner(learner    = learner,
                    fit_method = learner.fit_one_cycle(2, 1e-3),#lr
                    get_metric = learner.tracker.best,
                    n_trials   = 3
                   )

epoch,train_loss,valid_loss,accuracy,time
0,0.575656,0.600804,0.780769,00:00
1,0.468697,0.506795,0.796154,00:00


0.7961538434028625


epoch,train_loss,valid_loss,accuracy,time
0,0.492658,0.478840,0.765385,00:00
1,0.415909,0.412276,0.788462,00:00


0.7884615659713745


epoch,train_loss,valid_loss,accuracy,time
0,0.588335,0.612093,0.757692,00:00
1,0.476127,0.507247,0.788462,00:00


0.7884615659713745


In [32]:
#export
print("Number of finished trials: {}".format(len(study.trials)))

Number of finished trials: 3


In [33]:
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))

Best trial:
  Value: 0.7961538434028625


In [34]:
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

  Params: 
    fill_strategy: median
    toggle_Normalize: True
    mean: 7.803966587331513
    opt_func: Adam
    n_layers: 1
    layer_size_1: 150
    moms: 0.9044101813053794
    wd_bn_bias: False


In [35]:
study.trials[0].params

{'fill_strategy': 'median',
 'toggle_Normalize': True,
 'mean': 7.803966587331513,
 'opt_func': 'Adam',
 'n_layers': 1,
 'layer_size_1': 150,
 'moms': 0.9044101813053794,
 'wd_bn_bias': False}

In [36]:
#export
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,params_fill_strategy,params_layer_size_1,params_layer_size_2,params_layer_size_3,params_mean,params_moms,params_n_layers,params_opt_func,params_toggle_Normalize,params_wd_bn_bias,state
0,0,0.796154,2020-06-21 00:12:17.241241,2020-06-21 00:12:18.097564,median,150,NaN,NaN,7.803967,0.904410,1,Adam,True,False,COMPLETE
1,1,0.788462,2020-06-21 00:12:18.097564,2020-06-21 00:12:19.381564,median,100,300.0,300.0,NaN,0.937827,3,QHAdam,False,True,COMPLETE
2,2,0.788462,2020-06-21 00:12:19.382564,2020-06-21 00:12:20.296565,median,100,50.0,NaN,NaN,0.984989,2,Adam,False,True,COMPLETE


In [37]:
from nbdev.export import notebook2script

In [38]:
notebook2script()

Converted 00_core.ipynb.
Converted 01_api.ipynb.
Converted 02_oo_hp.ipynb.
Converted 03_hypster_prepare.ipynb.
Converted 04_tabular_api.ipynb.
Converted 05_sklearn.ipynb.
Converted fastai_adult_tutorial.ipynb.
Converted index.ipynb.
